In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import random
import os
import math

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 42
seed_everything(SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {DEVICE}")

# ======================================================
# [V11-Improved] MDN 개선 버전
# ======================================================
BATCH_SIZE = 64
LR_BASE = 1e-3  # ✅ 5e-4 → 1e-3
WARMUP_EPOCHS = 3  # ✅ 5 → 3
EPOCHS_BASE = 50
DROPOUT = 0.2
MAX_SEQ_LEN = 30
GRAD_CLIP = 1.0

HIDDEN_DIM = 256
LSTM_LAYERS = 3
BIDIRECTIONAL = True

# MDN 개선 파라미터
NUM_GAUSSIANS = 2  # ✅ 3 → 2 (주요/대안)
MIN_SIGMA = 0.01   # ✅ 더 큰 하한
MAX_SIGMA = 0.3    # ✅ 분산 상한 추가
HYBRID_LOSS_WEIGHT = 0.3  # ✅ Hybrid Loss 가중치

print(f"[V11-Improved] MDN 개선 버전")
print(f"  개선사항:")
print(f"  1. Gaussians: 3 → 2 (단순화)")
print(f"  2. LR: 5e-4 → 1e-3 (V10 수준)")
print(f"  3. Sigma 범위 제한: [0.01, 0.3]")
print(f"  4. Hybrid Loss: NLL + 0.3*MSE")
print(f"  5. 예측 전략: 가중평균 (mean)")

Device: cuda
[V11-Improved] MDN 개선 버전
  개선사항:
  1. Gaussians: 3 → 2 (단순화)
  2. LR: 5e-4 → 1e-3 (V10 수준)
  3. Sigma 범위 제한: [0.01, 0.3]
  4. Hybrid Loss: NLL + 0.3*MSE
  5. 예측 전략: 가중평균 (mean)


In [ ]:
# 데이터 로드 (V11과 동일)
BASE_DIR = "./open_track1"
if not os.path.exists(BASE_DIR): BASE_DIR = "."

TRAIN_PATH = os.path.join(BASE_DIR, "train.csv")
TEST_META_PATH = os.path.join(BASE_DIR, "test.csv")
MATCH_PATH = os.path.join(BASE_DIR, "match_info.csv")

train_df = pd.read_csv(TRAIN_PATH)
print(f"✅ Train Loaded: {train_df.shape}")

if os.path.exists(TEST_META_PATH):
    test_meta = pd.read_csv(TEST_META_PATH)
    print(f"ℹ️ Reading {len(test_meta)} test files...")
    test_dfs = []
    for _, row in tqdm(test_meta.iterrows(), total=len(test_meta), desc="Loading Test CSVs"):
        rel_path = row['path']
        paths_to_try = [
            rel_path,
            os.path.join(BASE_DIR, rel_path.lstrip("./")),
            os.path.join(BASE_DIR, "test", str(row['game_id']), os.path.basename(rel_path))
        ]
        for p in paths_to_try:
            if os.path.exists(p):
                test_dfs.append(pd.read_csv(p))
                break
    if test_dfs:
        test_df = pd.concat(test_dfs, ignore_index=True)
        print(f"✅ Test Data Merged: {test_df.shape}")
else:
    raise FileNotFoundError("test.csv not found")

if os.path.exists(MATCH_PATH):
    match_info = pd.read_csv(MATCH_PATH)
    match_subset = match_info[['game_id', 'home_team_id', 'venue']]
    train_df = pd.merge(train_df, match_subset, on='game_id', how='left')
    test_df = pd.merge(test_df, match_subset, on='game_id', how='left')

def preprocess(df):
    if 'home_team_id' in df.columns:
        df['is_home'] = (df['team_id'] == df['home_team_id']).astype(float)
    else:
        df['is_home'] = 0.5
    if 'end_x' not in df.columns:
        df['end_x'] = 0.0
        df['end_y'] = 0.0
    else:
        df['end_x'] = df['end_x'].fillna(0.0)
        df['end_y'] = df['end_y'].fillna(0.0)
    return df

train_df = preprocess(train_df)
test_df = preprocess(test_df)

ID_COL = 'game_episode' if 'game_episode' in train_df.columns else 'episode_id'
print(f"Data Ready. ID Column: {ID_COL}")

✅ Train Loaded: (356721, 15)
ℹ️ Reading 2414 test files...


Loading Test CSVs: 100%|██████████| 2414/2414 [00:04<00:00, 527.01it/s]


✅ Test Data Merged: (53110, 15)
Data Ready. ID Column: game_episode


In [ ]:
# 피처 엔지니어링 (V11과 동일)
TOP_TYPES = ['Pass', 'Carry', 'Recovery', 'Interception', 'Duel', 'Tackle', 
             'Throw-In', 'Clearance', 'Intervention', 'Block', 'Pass_Freekick', 
             'Cross', 'Goal Kick', 'Error', 'Shot']
ALL_RESULTS = ['Successful', 'Unsuccessful', 'On Target', 'Yellow_Card', 
               'Blocked', 'Keeper Rush-Out', 'Low Quality Shot', 'Off Target']

def make_features(group):
    n = len(group)
    sx = group['start_x'].values / 105.0
    sy = group['start_y'].values / 68.0
    ex = group['end_x'].values / 105.0
    ey = group['end_y'].values / 68.0
    is_home = group['is_home'].values
    
    if 'time_seconds' in group.columns:
        times = group['time_seconds'].values
        dt = np.zeros(n, dtype=np.float32)
        dt[1:] = times[1:] - times[:-1]
        dt = np.maximum(dt, 0.1)
    else:
        dt = np.ones(n, dtype=np.float32)

    dx = ex - sx
    dy = ey - sy
    dist_meter = np.sqrt((dx*105)**2 + (dy*68)**2)
    cumsum_dx = np.cumsum(dx) / 105.0
    cumsum_dy = np.cumsum(dy) / 68.0
    lag_dist_m = np.roll(dist_meter, 1); lag_dist_m[0] = 0
    lag_cumsum_dx = np.roll(cumsum_dx, 1); lag_cumsum_dx[0] = 0
    lag_cumsum_dy = np.roll(cumsum_dy, 1); lag_cumsum_dy[0] = 0
    lag_dt = np.roll(dt, 1); lag_dt[0] = 1.0
    lag_speed = lag_dist_m / np.maximum(lag_dt, 0.1)
    
    if 'player_id' in group.columns:
        p_ids = group['player_id'].values
        is_same = np.zeros(n, dtype=np.float32)
        is_same[1:] = (p_ids[1:] == p_ids[:-1]).astype(np.float32)
    else:
        is_same = np.zeros(n, dtype=np.float32)

    progress = np.arange(n) / max(n-1, 1)
    is_second_half = (group['period_id'].values > 1).astype(np.float32) if 'period_id' in group.columns else np.zeros(n)
    
    GOAL_X, GOAL_Y = 105.0, 34.0
    sx_real, sy_real = sx * 105.0, sy * 68.0
    dist_to_goal = np.sqrt((sx_real - GOAL_X)**2 + (sy_real - GOAL_Y)**2) / 105.0
    angle_to_goal = np.arctan2(GOAL_Y - sy_real, GOAL_X - sx_real)
    angle_sin, angle_cos = np.sin(angle_to_goal), np.cos(angle_to_goal)
    dist_to_sideline = np.minimum(sy_real, 68.0 - sy_real) / 68.0
    dist_to_endline = np.minimum(sx_real, 105.0 - sx_real) / 105.0
    
    def get_zone(x_norm):
        if x_norm < 35.0/105.0: return 0
        elif x_norm < 70.0/105.0: return 1
        else: return 2
    
    zones = np.array([get_zone(x) for x in sx])
    zone_onehot = np.zeros((n, 3), dtype=np.float32)
    for i, z in enumerate(zones): zone_onehot[i, z] = 1.0
    
    types_onehot = np.zeros((n, len(TOP_TYPES) + 1), dtype=np.float32)
    for i, t in enumerate(group['type_name'].values):
        types_onehot[i, TOP_TYPES.index(t) if t in TOP_TYPES else -1] = 1.0
    
    results_onehot = np.zeros((n, len(ALL_RESULTS) + 1), dtype=np.float32)
    for i, r in enumerate(group['result_name'].values):
        results_onehot[i, ALL_RESULTS.index(r) if r in ALL_RESULTS else -1] = 1.0

    features = []
    for i in range(n):
        scalars = [sx[i], sy[i], lag_cumsum_dx[i], lag_cumsum_dy[i], lag_dist_m[i]/100.0,
                   lag_speed[i]/10.0, dt[i]/10.0, progress[i], is_home[i], is_same[i],
                   is_second_half[i], dist_to_goal[i], angle_sin[i], angle_cos[i],
                   dist_to_sideline[i], dist_to_endline[i]]
        feat_vec = np.concatenate([scalars, zone_onehot[i], types_onehot[i], results_onehot[i]])
        features.append(feat_vec)
        
        if i < n - 1:
            ex_real, ey_real = ex[i] * 105.0, ey[i] * 68.0
            end_dist_to_goal = np.sqrt((ex_real - GOAL_X)**2 + (ey_real - GOAL_Y)**2) / 105.0
            end_angle = np.arctan2(GOAL_Y - ey_real, GOAL_X - ex_real)
            scalars_end = scalars.copy()
            scalars_end[0:2] = [ex[i], ey[i]]
            scalars_end[2:4] = [cumsum_dx[i], cumsum_dy[i]]
            scalars_end[11:16] = [end_dist_to_goal, np.sin(end_angle), np.cos(end_angle),
                                   min(ey_real, 68.0 - ey_real) / 68.0,
                                   min(ex_real, 105.0 - ex_real) / 105.0]
            end_zone_onehot = np.zeros(3, dtype=np.float32)
            end_zone_onehot[get_zone(ex[i])] = 1.0
            feat_vec_end = np.concatenate([scalars_end, end_zone_onehot, types_onehot[i], results_onehot[i]])
            features.append(feat_vec_end)
            
    return np.array(features, dtype=np.float32)

dummy_group = train_df.iloc[:5].copy()
INPUT_DIM = make_features(dummy_group).shape[1]
print(f"✅ Input Dimension: {INPUT_DIM}")

✅ Input Dimension: 44


In [ ]:
# 데이터셋 (V11과 동일)
class SoccerDataset(Dataset):
    def __init__(self, df, mode='train'):
        self.mode = mode
        self.episodes, self.targets, self.team_ids, self.episode_ids = [], [], [], []
        for name, group in tqdm(df.groupby(ID_COL, sort=False), desc=f"Dataset ({mode})"):
            if mode == 'train' and len(group) < 2: continue
            seq = make_features(group)
            team_id = group.iloc[0]['team_id']
            if mode == 'train':
                last = group.iloc[-1]
                self.targets.append([last['end_x']/105.0, last['end_y']/68.0])
                self.episodes.append(seq)
                self.team_ids.append(team_id)
            else:
                self.episodes.append(seq)
                self.team_ids.append(team_id)
                self.episode_ids.append(str(name))

    def __len__(self): return len(self.episodes)
    def __getitem__(self, idx):
        seq = torch.FloatTensor(self.episodes[idx])
        if len(seq) > MAX_SEQ_LEN: seq = seq[-MAX_SEQ_LEN:]
        if self.mode == 'train':
            return seq, torch.FloatTensor(self.targets[idx]), self.team_ids[idx]
        return seq, self.team_ids[idx], self.episode_ids[idx]

def collate_fn(batch):
    seqs = [b[0] for b in batch]
    lens = torch.LongTensor([len(s) for s in seqs])
    padded = pad_sequence(seqs, batch_first=True, padding_value=0)
    mask = torch.arange(padded.size(1))[None, :] >= lens[:, None]
    if isinstance(batch[0][2], str):
        return padded, mask, lens, torch.LongTensor([b[1] for b in batch]), [b[2] for b in batch]
    return padded, torch.stack([b[1] for b in batch]), mask, lens, torch.LongTensor([b[2] for b in batch])

full_dataset = SoccerDataset(train_df, mode='train')
test_dataset = SoccerDataset(test_df, mode='test')
print(f"✅ Dataset: {len(full_dataset)} train, {len(test_dataset)} test")

Dataset (test): 100%|██████████| 2414/2414 [00:02<00:00, 864.90it/s]

✅ Dataset: 15428 train, 2414 test


In [ ]:
# ======================================================
# [V11-Improved] MDN 모델 (개선)
# ======================================================

class SpatialAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.goal_attn = nn.Sequential(nn.Linear(3, 16), nn.ReLU(), nn.Linear(16, 1))
        self.zone_attn = nn.Sequential(nn.Linear(3, 8), nn.ReLU(), nn.Linear(8, 1))
        self.pos_attn = nn.Sequential(nn.Linear(2, 8), nn.ReLU(), nn.Linear(8, 1))
        self.fusion = nn.Linear(3, 1)
    def forward(self, x):
        return self.fusion(torch.cat([self.pos_attn(x[..., 0:2]), 
                                       self.goal_attn(x[..., 11:14]), 
                                       self.zone_attn(x[..., 16:19])], dim=-1))

class TemporalAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.pos_encoding = nn.Parameter(torch.randn(1, 100, hidden_dim) * 0.02)
        self.temporal_attn = nn.Sequential(nn.Linear(hidden_dim, hidden_dim // 2), 
                                           nn.Tanh(), nn.Dropout(0.1), 
                                           nn.Linear(hidden_dim // 2, 1))
    def forward(self, lstm_out):
        return self.temporal_attn(lstm_out + self.pos_encoding[:, :lstm_out.size(1), :])

class SpatialTemporalFusion(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.spatial_weight = nn.Parameter(torch.tensor(0.5))
        self.temporal_weight = nn.Parameter(torch.tensor(0.5))
        self.combine = nn.Sequential(nn.Linear(2, 8), nn.ReLU(), nn.Linear(8, 1), nn.Sigmoid())
    def forward(self, s, t):
        return (torch.sigmoid(self.spatial_weight) * s + 
                torch.sigmoid(self.temporal_weight) * t) * self.combine(torch.cat([s, t], -1))

class ImprovedMDNPredictor(nn.Module):
    """
    [V11-Improved] 개선된 MDN
    - Gaussian 수 감소 (2개)
    - Sigma 범위 제한
    - 초기화 개선
    """
    def __init__(self, input_dim, hidden_dim, num_layers, dropout, num_gaussians=2, bidirectional=True):
        super().__init__()
        self.num_gaussians = num_gaussians
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True,
                           dropout=dropout if num_layers > 1 else 0, bidirectional=bidirectional)
        lstm_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        
        self.spatial_attn = SpatialAttention(lstm_output_dim)
        self.temporal_attn = TemporalAttention(lstm_output_dim)
        self.fusion = SpatialTemporalFusion(lstm_output_dim)
        
        # MDN Heads (더 강력한 초기화)
        self.pi_head = nn.Sequential(
            nn.Linear(lstm_output_dim, lstm_output_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_output_dim // 2, num_gaussians)
        )
        self.mu_head = nn.Sequential(
            nn.Linear(lstm_output_dim, lstm_output_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_output_dim // 2, num_gaussians * 2)
        )
        self.sigma_head = nn.Sequential(
            nn.Linear(lstm_output_dim, lstm_output_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_output_dim // 2, num_gaussians * 2)
        )
        
        # ✅ Mu 초기화: 중앙 근처 bias
        nn.init.constant_(self.mu_head[-1].bias, 0.5)
        nn.init.xavier_uniform_(self.mu_head[-1].weight, gain=0.1)

    def forward(self, x, mask=None, lengths=None):
        batch_size, seq_len = x.size(0), x.size(1)
        
        # LSTM + Attention
        if lengths is not None:
            packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
            lstm_out, _ = self.lstm(packed)
            lstm_out, _ = pad_packed_sequence(lstm_out, batch_first=True, total_length=seq_len)
        else:
            lstm_out, _ = self.lstm(x)
        
        fused_attn = self.fusion(self.spatial_attn(x), self.temporal_attn(lstm_out))
        if mask is not None:
            fused_attn = fused_attn.masked_fill(mask.unsqueeze(-1), float('-inf'))
        final_attn = torch.softmax(fused_attn, dim=1)
        context = torch.sum(lstm_out * final_attn, dim=1)
        
        # MDN 출력
        pi = torch.softmax(self.pi_head(context), dim=1)
        mu = torch.sigmoid(self.mu_head(context)).view(batch_size, self.num_gaussians, 2)  # ✅ Sigmoid로 0~1 강제
        
        # ✅ Sigma 범위 제한: [MIN_SIGMA, MAX_SIGMA]
        sigma_raw = self.sigma_head(context).view(batch_size, self.num_gaussians, 2)
        sigma = torch.sigmoid(sigma_raw) * (MAX_SIGMA - MIN_SIGMA) + MIN_SIGMA
        
        return pi, mu, sigma


def hybrid_mdn_loss(pi, mu, sigma, target, mse_weight=HYBRID_LOSS_WEIGHT):
    """
    ✅ Hybrid Loss: NLL + MSE
    - NLL: 확률 분포 학습
    - MSE: 직접적인 좌표 학습 (안정성)
    """
    batch_size = target.size(0)
    target_expanded = target.unsqueeze(1).expand_as(mu)
    
    # NLL Loss
    diff = target_expanded - mu
    log_prob_components = (
        -0.5 * math.log(2 * math.pi) - torch.log(sigma) - 0.5 * (diff / sigma) ** 2
    )
    log_prob = log_prob_components.sum(dim=2)
    weighted_log_prob = log_prob + torch.log(pi + 1e-8)
    nll_loss = -torch.logsumexp(weighted_log_prob, dim=1).mean()
    
    # ✅ MSE Loss (가중 평균 예측)
    pred_mean = (pi.unsqueeze(-1) * mu).sum(dim=1)
    mse_loss = nn.functional.mse_loss(pred_mean, target)
    
    # Hybrid
    total_loss = nll_loss + mse_weight * mse_loss
    
    return total_loss, nll_loss, mse_loss


def mdn_predict_improved(pi, mu, sigma, strategy='mean'):
    """
    ✅ 개선된 예측: 기본 전략을 mean으로 변경
    - mean: 더 안정적이고 모든 옵션 고려
    """
    if strategy == 'mode':
        max_idx = torch.argmax(pi, dim=1)
        pred = mu[torch.arange(len(mu)), max_idx]
    elif strategy == 'mean':
        pred = (pi.unsqueeze(-1) * mu).sum(dim=1)
    else:
        raise ValueError(f"Unknown strategy: {strategy}")
    
    return torch.clamp(pred, 0.0, 1.0)


model = ImprovedMDNPredictor(INPUT_DIM, HIDDEN_DIM, LSTM_LAYERS, DROPOUT, NUM_GAUSSIANS, BIDIRECTIONAL).to(DEVICE)

print("="*70)
print("✅ [V11-Improved] MDN 모델 준비 완료")
print("="*70)
print(f"주요 개선사항:")
print(f"  1. Gaussians: 2개 (단순화)")
print(f"  2. Mu: Sigmoid로 0~1 보장")
print(f"  3. Sigma: [{MIN_SIGMA}, {MAX_SIGMA}] 제한")
print(f"  4. Loss: Hybrid (NLL + {HYBRID_LOSS_WEIGHT}*MSE)")
print(f"  5. 예측: mean (가중평균)")
print(f"  6. LR: 1e-3 (V10 수준)")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
print("="*70)

✅ [V11-Improved] MDN 모델 준비 완료
주요 개선사항:
  1. Gaussians: 2개 (단순화)
  2. Mu: Sigmoid로 0~1 보장
  3. Sigma: [0.01, 0.3] 제한
  4. Loss: Hybrid (NLL + 0.3*MSE)
  5. 예측: mean (가중평균)
  6. LR: 1e-3 (V10 수준)
  Parameters: 4,351,949


In [ ]:
# ======================================================
# 🔍 Bayesian Optimization + 최적 파라미터로 학습
# ======================================================
import optuna
from torch.optim.lr_scheduler import CosineAnnealingLR
import json
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("🔍 Step 1: Bayesian Optimization으로 최적 하이퍼파라미터 찾기")
print("="*70)

# ======================================================
# OPTUNA OBJECTIVE FUNCTION
# ======================================================
def objective(trial):
    """하이퍼파라미터 튜닝"""
    # 1. 하이퍼파라미터 샘플링
    num_gaussians = trial.suggest_int("num_gaussians", 2, 5)
    hidden_dim = trial.suggest_categorical("hidden_dim", [128, 256, 512])
    lstm_layers = trial.suggest_int("lstm_layers", 2, 4)
    dropout = trial.suggest_float("dropout", 0.1, 0.4)
    lr = trial.suggest_float("lr", 5e-4, 5e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    min_sigma = trial.suggest_float("min_sigma", 0.005, 0.02)
    max_sigma = trial.suggest_float("max_sigma", 0.2, 0.5)
    hybrid_loss_weight = trial.suggest_float("hybrid_loss_weight", 0.1, 0.5)
    
    # 2. 데이터 로더
    seed_everything(42)
    train_idx, val_idx = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=42)
    train_loader = DataLoader(torch.utils.data.Subset(full_dataset, train_idx), 
                              batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(torch.utils.data.Subset(full_dataset, val_idx), 
                           batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    
    # 3. 모델 생성
    model_trial = ImprovedMDNPredictor(
        INPUT_DIM, hidden_dim, lstm_layers, dropout, num_gaussians, BIDIRECTIONAL
    ).to(DEVICE)
    
    optimizer = optim.AdamW(model_trial.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-6)
    
    # 4. 빠른 학습 (20 epochs)
    OPTUNA_EPOCHS = 20
    best_dist = float('inf')
    patience, patience_counter = 5, 0
    
    for epoch in range(OPTUNA_EPOCHS):
        model_trial.train()
        for seqs, targets, mask, lens, _ in train_loader:
            seqs, targets, mask, lens = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
            optimizer.zero_grad()
            pi, mu, sigma = model_trial(seqs, mask, lens)
            sigma = torch.clamp(sigma, min_sigma, max_sigma)
            
            # Loss
            target_expanded = targets.unsqueeze(1).expand_as(mu)
            diff = target_expanded - mu
            log_prob_components = -0.5 * math.log(2 * math.pi) - torch.log(sigma) - 0.5 * (diff / sigma) ** 2
            log_prob = log_prob_components.sum(dim=2)
            weighted_log_prob = log_prob + torch.log(pi + 1e-8)
            nll_loss = -torch.logsumexp(weighted_log_prob, dim=1).mean()
            pred_mean = (pi.unsqueeze(-1) * mu).sum(dim=1)
            mse_loss = nn.functional.mse_loss(pred_mean, targets)
            loss = nll_loss + hybrid_loss_weight * mse_loss
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model_trial.parameters(), GRAD_CLIP)
            optimizer.step()
        
        # Validation
        model_trial.eval()
        dists = []
        with torch.no_grad():
            for seqs, targets, mask, lens, _ in val_loader:
                seqs, targets, mask, lens = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
                pi, mu, sigma = model_trial(seqs, mask, lens)
                pred = mdn_predict_improved(pi, mu, sigma, strategy='mean')
                p_real = pred.cpu().numpy() * np.array([105.0, 68.0])
                t_real = targets.cpu().numpy() * np.array([105.0, 68.0])
                dists.extend(np.sqrt(np.sum((p_real - t_real)**2, axis=1)))
        
        avg_dist = np.mean(dists)
        scheduler.step()
        
        # Early stopping
        if avg_dist < best_dist:
            best_dist = avg_dist
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break
        
        # Optuna pruning
        trial.report(avg_dist, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return best_dist


# ======================================================
# OPTUNA 실행
# ======================================================
print("\n📊 탐색 공간:")
print("  - num_gaussians: 2~5, hidden_dim: [128,256,512], lstm_layers: 2~4")
print("  - dropout: 0.1~0.4, lr: 5e-4~5e-3, batch_size: [32,64,128]")
print("  - min_sigma: 0.005~0.02, max_sigma: 0.2~0.5, hybrid_weight: 0.1~0.5")
print("\n⏱️  예상 시간: 2-3시간 (30 trials x 5분)")
print("="*70 + "\n")

study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
    sampler=optuna.samplers.TPESampler(seed=42)
)

study.optimize(objective, n_trials=30, show_progress_bar=True)

# ======================================================
# 결과 출력
# ======================================================
print("\n" + "="*70)
print("✅ Optuna 최적화 완료!")
print("="*70)
print(f"\n🏆 최고 성능: {study.best_value:.4f}m")
print(f"\n📌 최적 하이퍼파라미터:")
for key, value in study.best_params.items():
    print(f"  - {key}: {value}")

# 저장
with open('best_hyperparameters.json', 'w', encoding='utf-8') as f:
    json.dump({'best_value': study.best_value, 'best_params': study.best_params, 
               'n_trials': len(study.trials)}, f, indent=2, ensure_ascii=False)

# ======================================================
# Step 2: 최적 파라미터로 최종 학습
# ======================================================
print("\n" + "="*70)
print("🚀 Step 2: 최적 하이퍼파라미터로 최종 학습 (50 epochs)")
print("="*70)

best_params = study.best_params
SEEDS = [42, 2024, 777]
PRED_STRATEGY = 'mean'
all_histories = []

for i, seed in enumerate(SEEDS):
    print(f"\n{'='*70}")
    print(f"📦 [모델 {i+1}/3] Seed {seed}")
    print(f"{'='*70}")
    seed_everything(seed)
    
    train_idx, val_idx = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=seed)
    train_loader = DataLoader(torch.utils.data.Subset(full_dataset, train_idx), 
                              batch_size=best_params['batch_size'], shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(torch.utils.data.Subset(full_dataset, val_idx), 
                           batch_size=best_params['batch_size'], shuffle=False, collate_fn=collate_fn)
    
    # 최적 파라미터로 모델 생성
    model = ImprovedMDNPredictor(
        INPUT_DIM, best_params['hidden_dim'], best_params['lstm_layers'], 
        best_params['dropout'], best_params['num_gaussians'], BIDIRECTIONAL
    ).to(DEVICE)
    
    optimizer = optim.AdamW(model.parameters(), lr=best_params['lr'], weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS_BASE - WARMUP_EPOCHS, eta_min=1e-6)
    
    best_dist = float('inf')
    history = {'train_loss': [], 'train_nll': [], 'train_mse': [], 'val_dist': [], 'lr': []}
    
    for epoch in range(EPOCHS_BASE):
        # Warmup
        if epoch < WARMUP_EPOCHS:
            for param_group in optimizer.param_groups:
                param_group['lr'] = best_params['lr'] * (epoch + 1) / WARMUP_EPOCHS
        
        # Training
        model.train()
        train_loss, train_nll, train_mse = 0.0, 0.0, 0.0
        for seqs, targets, mask, lens, _ in train_loader:
            seqs, targets, mask, lens = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
            optimizer.zero_grad()
            pi, mu, sigma = model(seqs, mask, lens)
            sigma = torch.clamp(sigma, best_params['min_sigma'], best_params['max_sigma'])
            
            # Loss
            target_expanded = targets.unsqueeze(1).expand_as(mu)
            diff = target_expanded - mu
            log_prob_components = -0.5 * math.log(2 * math.pi) - torch.log(sigma) - 0.5 * (diff / sigma) ** 2
            log_prob = log_prob_components.sum(dim=2)
            weighted_log_prob = log_prob + torch.log(pi + 1e-8)
            nll_loss = -torch.logsumexp(weighted_log_prob, dim=1).mean()
            pred_mean = (pi.unsqueeze(-1) * mu).sum(dim=1)
            mse_loss = nn.functional.mse_loss(pred_mean, targets)
            loss = nll_loss + best_params['hybrid_loss_weight'] * mse_loss
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            optimizer.step()
            train_loss += loss.item()
            train_nll += nll_loss.item()
            train_mse += mse_loss.item()
        
        # Validation
        model.eval()
        dists = []
        with torch.no_grad():
            for seqs, targets, mask, lens, _ in val_loader:
                seqs, targets, mask, lens = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
                pi, mu, sigma = model(seqs, mask, lens)
                pred = mdn_predict_improved(pi, mu, sigma, strategy=PRED_STRATEGY)
                p_real = pred.cpu().numpy() * np.array([105.0, 68.0])
                t_real = targets.cpu().numpy() * np.array([105.0, 68.0])
                dists.extend(np.sqrt(np.sum((p_real - t_real)**2, axis=1)))
        
        avg_dist = np.mean(dists)
        avg_loss = train_loss / len(train_loader)
        avg_nll = train_nll / len(train_loader)
        avg_mse = train_mse / len(train_loader)
        
        if epoch >= WARMUP_EPOCHS:
            scheduler.step()
        
        current_lr = optimizer.param_groups[0]['lr']
        history['train_loss'].append(avg_loss)
        history['train_nll'].append(avg_nll)
        history['train_mse'].append(avg_mse)
        history['val_dist'].append(avg_dist)
        history['lr'].append(current_lr)
        
        if (epoch + 1) % 5 == 0 or epoch < 5:
            print(f"  [Epoch {epoch+1:2d}/{EPOCHS_BASE}] "
                  f"Loss: {avg_loss:.4f} (NLL: {avg_nll:.4f}, MSE: {avg_mse:.4f}) | "
                  f"Val: {avg_dist:.4f}m | LR: {current_lr:.6f}")
        
        if avg_dist < best_dist:
            best_dist = avg_dist
            torch.save(model.state_dict(), f'optimized_mdn_{i}.pth')
    
    history['best_dist'] = best_dist
    history['seed'] = seed
    all_histories.append(history)
    print(f"  ✅ Best: {best_dist:.4f}m")

print(f"\n{'='*70}")
print(f"✅ 최종 학습 완료!")
print(f"{'='*70}")
print(f"평균 Best: {np.mean([h['best_dist'] for h in all_histories]):.4f}m")
print(f"최고 성능: {min([h['best_dist'] for h in all_histories]):.4f}m")
print(f"{'='*70}")

In [ ]:
# ======================================================
# 추론 (최적화된 모델 사용)
# ======================================================
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

models = []
for i in range(3):
    model = ImprovedMDNPredictor(
        INPUT_DIM, 
        best_params['hidden_dim'], 
        best_params['lstm_layers'], 
        best_params['dropout'], 
        best_params['num_gaussians'], 
        BIDIRECTIONAL
    ).to(DEVICE)
    model.load_state_dict(torch.load(f'optimized_mdn_{i}.pth'))  # ✅ 최적화된 모델 로드
    model.eval()
    models.append(model)

results = []
with torch.no_grad():
    for seqs, mask, lens, team_ids, episode_ids in tqdm(test_loader, desc="Inference"):
        seqs, mask, lens = seqs.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
        preds = []
        for model in models:
            pi, mu, sigma = model(seqs, mask, lens)
            pred = mdn_predict_improved(pi, mu, sigma, strategy='mean')
            preds.append(pred.cpu().numpy())
        avg_pred = np.mean(preds, axis=0)
        for i, eid in enumerate(episode_ids):
            results.append({'game_episode': eid, 'pred_x': avg_pred[i, 0] * 105.0, 
                          'pred_y': avg_pred[i, 1] * 68.0})

pred_df = pd.DataFrame(results)
SUBMISSION_PATH = "open_track1/sample_submission.csv"
if os.path.exists(SUBMISSION_PATH):
    sub = pd.read_csv(SUBMISSION_PATH)
else:
    sub = pd.read_csv(TEST_META_PATH).rename(columns={'episode_id': 'game_episode'})[['game_episode']]

final_sub = pd.merge(sub[['game_episode']], pred_df, on='game_episode', how='left')
final_sub = final_sub.rename(columns={'pred_x': 'end_x', 'pred_y': 'end_y'})
if final_sub.isnull().sum().sum() > 0:
    final_sub = final_sub.fillna(50.0)

filename = "optimized_submission.csv"
final_sub.to_csv(filename, index=False)

print(f"\n✅ 제출 파일: {filename}")
print(f"📌 사용된 최적 파라미터:")
for key, value in best_params.items():
    print(f"  - {key}: {value}")
print(final_sub.head())

In [ ]:
# 시각화
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('[V11-Improved] MDN 학습 곡선', fontsize=16, fontweight='bold')

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

# 1. Hybrid Loss
ax1 = axes[0, 0]
for i, h in enumerate(all_histories):
    epochs = range(1, len(h['train_loss']) + 1)
    ax1.plot(epochs, h['train_loss'], label=f"Model {i+1}", color=colors[i], linewidth=2)
ax1.set_title('Training Loss (Hybrid: NLL + MSE)')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Validation Distance
ax2 = axes[0, 1]
for i, h in enumerate(all_histories):
    epochs = range(1, len(h['val_dist']) + 1)
    ax2.plot(epochs, h['val_dist'], label=f"Model {i+1}", color=colors[i], linewidth=2)
    best_idx = np.argmin(h['val_dist'])
    ax2.scatter(best_idx + 1, h['val_dist'][best_idx], color=colors[i], s=100, marker='*')
ax2.set_title(f'Validation Distance ({PRED_STRATEGY})')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Distance (m)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. NLL vs MSE
ax3 = axes[1, 0]
h = all_histories[0]
epochs = range(1, len(h['train_nll']) + 1)
ax3.plot(epochs, h['train_nll'], label='NLL', color='red', linewidth=2)
ax3_twin = ax3.twinx()
ax3_twin.plot(epochs, h['train_mse'], label='MSE', color='blue', linewidth=2)
ax3.set_xlabel('Epoch')
ax3.set_ylabel('NLL', color='red')
ax3_twin.set_ylabel('MSE', color='blue')
ax3.set_title('Loss Components (Model 1)')
ax3.grid(True, alpha=0.3)

# 4. Best Distances
ax4 = axes[1, 1]
best_dists = [h['best_dist'] for h in all_histories]
bars = ax4.bar(range(len(best_dists)), best_dists, color=colors)
ax4.set_xticks(range(len(best_dists)))
ax4.set_xticklabels([f'Model {i+1}' for i in range(len(best_dists))])
ax4.set_ylabel('Best Distance (m)')
ax4.set_title('Best Performance')
for bar, dist in zip(bars, best_dists):
    ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height(), 
             f'{dist:.4f}m', ha='center', va='bottom', fontweight='bold')
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('v11_improved_mdn_history.png', dpi=150)
plt.show()

print(f"\n최종 결과:")
for i, h in enumerate(all_histories):
    print(f"  Model {i+1}: {h['best_dist']:.4f}m")
print(f"  평균: {np.mean(best_dists):.4f}m")

In [ ]:
# ======================================================
# 🔍 Bayesian Optimization으로 하이퍼파라미터 튜닝
# ======================================================
import optuna
import json
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("🔍 Bayesian Optimization 하이퍼파라미터 튜닝 시작")
print("="*70)

# Optuna Objective Function
def objective(trial):
    """
    Optuna가 최적화할 목적 함수
    Trial마다 다른 하이퍼파라미터로 모델을 학습하고 validation 성능 반환
    """
    
    # ======================================================
    # 1. 하이퍼파라미터 샘플링
    # ======================================================
    num_gaussians = trial.suggest_int("num_gaussians", 2, 5)
    hidden_dim = trial.suggest_categorical("hidden_dim", [128, 256, 512])
    lstm_layers = trial.suggest_int("lstm_layers", 2, 4)
    dropout = trial.suggest_float("dropout", 0.1, 0.4)
    lr = trial.suggest_float("lr", 5e-4, 5e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    min_sigma = trial.suggest_float("min_sigma", 0.005, 0.02)
    max_sigma = trial.suggest_float("max_sigma", 0.2, 0.5)
    hybrid_loss_weight = trial.suggest_float("hybrid_loss_weight", 0.1, 0.5)
    
    # ======================================================
    # 2. 모델 생성 (샘플링된 하이퍼파라미터 사용)
    # ======================================================
    seed_everything(42)  # 고정 seed로 공정한 비교
    
    # Train/Val split
    train_idx, val_idx = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=42)
    train_loader = DataLoader(torch.utils.data.Subset(full_dataset, train_idx), 
                              batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(torch.utils.data.Subset(full_dataset, val_idx), 
                           batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    
    # 모델 생성
    model_trial = ImprovedMDNPredictor(
        INPUT_DIM, hidden_dim, lstm_layers, dropout, 
        num_gaussians, BIDIRECTIONAL
    ).to(DEVICE)
    
    optimizer = optim.AdamW(model_trial.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-6)
    
    # ======================================================
    # 3. 학습 (빠른 평가를 위해 epochs 줄임)
    # ======================================================
    OPTUNA_EPOCHS = 20  # Trial마다 20 epochs만 (빠른 평가)
    best_dist = float('inf')
    patience = 5
    patience_counter = 0
    
    for epoch in range(OPTUNA_EPOCHS):
        # Training
        model_trial.train()
        train_loss = 0.0
        for seqs, targets, mask, lens, _ in train_loader:
            seqs, targets, mask, lens = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
            optimizer.zero_grad()
            pi, mu, sigma = model_trial(seqs, mask, lens)
            
            # Sigma 범위 제한 적용
            sigma = torch.clamp(sigma, min_sigma, max_sigma)
            
            # Loss 계산
            target_expanded = targets.unsqueeze(1).expand_as(mu)
            diff = target_expanded - mu
            log_prob_components = (
                -0.5 * math.log(2 * math.pi) - torch.log(sigma) - 0.5 * (diff / sigma) ** 2
            )
            log_prob = log_prob_components.sum(dim=2)
            weighted_log_prob = log_prob + torch.log(pi + 1e-8)
            nll_loss = -torch.logsumexp(weighted_log_prob, dim=1).mean()
            
            pred_mean = (pi.unsqueeze(-1) * mu).sum(dim=1)
            mse_loss = nn.functional.mse_loss(pred_mean, targets)
            
            loss = nll_loss + hybrid_loss_weight * mse_loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model_trial.parameters(), GRAD_CLIP)
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model_trial.eval()
        dists = []
        with torch.no_grad():
            for seqs, targets, mask, lens, _ in val_loader:
                seqs, targets, mask, lens = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
                pi, mu, sigma = model_trial(seqs, mask, lens)
                pred = mdn_predict_improved(pi, mu, sigma, strategy='mean')
                p_real = pred.cpu().numpy() * np.array([105.0, 68.0])
                t_real = targets.cpu().numpy() * np.array([105.0, 68.0])
                dists.extend(np.sqrt(np.sum((p_real - t_real)**2, axis=1)))
        
        avg_dist = np.mean(dists)
        scheduler.step()
        
        # Early stopping
        if avg_dist < best_dist:
            best_dist = avg_dist
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break
        
        # Optuna pruning (성능 안 좋은 trial 조기 종료)
        trial.report(avg_dist, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return best_dist


# ======================================================
# Optuna Study 실행
# ======================================================
print("\n📊 하이퍼파라미터 탐색 공간:")
print("  - num_gaussians: 2 ~ 5")
print("  - hidden_dim: [128, 256, 512]")
print("  - lstm_layers: 2 ~ 4")
print("  - dropout: 0.1 ~ 0.4")
print("  - learning_rate: 5e-4 ~ 5e-3")
print("  - batch_size: [32, 64, 128]")
print("  - min_sigma: 0.005 ~ 0.02")
print("  - max_sigma: 0.2 ~ 0.5")
print("  - hybrid_loss_weight: 0.1 ~ 0.5")
print("\n⏱️  예상 소요 시간: 약 2-3시간 (30 trials)")
print("="*70)

# Study 생성 및 실행
study = optuna.create_study(
    direction="minimize",  # validation distance 최소화
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5),  # 성능 안 좋은 trial 조기 종료
    sampler=optuna.samplers.TPESampler(seed=42)  # Bayesian Optimization
)

# 최적화 실행 (30 trials)
study.optimize(objective, n_trials=30, show_progress_bar=True)

In [ ]:
# ======================================================
# 📊 최적화 결과 분석 및 시각화
# ======================================================

print("\n" + "="*70)
print("✅ 최적화 완료!")
print("="*70)

print(f"\n🏆 최고 성능: {study.best_value:.4f}m")
print(f"\n📌 최적 하이퍼파라미터:")
for key, value in study.best_params.items():
    print(f"  - {key}: {value}")

print(f"\n📈 Trial 통계:")
print(f"  - 완료된 trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}")
print(f"  - Pruned trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")

# 상위 5개 trial
print(f"\n🔝 상위 5개 Trials:")
sorted_trials = sorted(study.trials, key=lambda t: t.value if t.value is not None else float('inf'))
for i, trial in enumerate(sorted_trials[:5]):
    print(f"  {i+1}. Trial {trial.number}: {trial.value:.4f}m")

# ======================================================
# 시각화
# ======================================================
fig = plt.figure(figsize=(15, 5))

# 1. Optimization History
ax1 = plt.subplot(1, 3, 1)
values = [t.value for t in study.trials if t.value is not None]
ax1.plot(values, marker='o', linestyle='-', alpha=0.6)
ax1.axhline(study.best_value, color='r', linestyle='--', label=f'Best: {study.best_value:.4f}m')
ax1.set_xlabel('Trial')
ax1.set_ylabel('Validation Distance (m)')
ax1.set_title('Optimization History')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Parameter Importances
try:
    importances = optuna.importance.get_param_importances(study)
    ax2 = plt.subplot(1, 3, 2)
    params = list(importances.keys())[:8]  # 상위 8개
    imp_values = [importances[p] for p in params]
    ax2.barh(params, imp_values)
    ax2.set_xlabel('Importance')
    ax2.set_title('Parameter Importances')
    ax2.grid(True, alpha=0.3, axis='x')
except Exception as e:
    print(f"Parameter importance 계산 실패: {e}")

# 3. Best vs Average per epoch
ax3 = plt.subplot(1, 3, 3)
all_intermediate = []
for trial in study.trials:
    if trial.state == optuna.trial.TrialState.COMPLETE:
        for step, value in trial.intermediate_values.items():
            all_intermediate.append((step, value))

if all_intermediate:
    from collections import defaultdict
    epoch_values = defaultdict(list)
    for step, value in all_intermediate:
        epoch_values[step].append(value)
    
    epochs = sorted(epoch_values.keys())
    avg_values = [np.mean(epoch_values[e]) for e in epochs]
    min_values = [np.min(epoch_values[e]) for e in epochs]
    
    ax3.plot(epochs, avg_values, label='Average', marker='o')
    ax3.plot(epochs, min_values, label='Best', marker='s')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Validation Distance (m)')
    ax3.set_title('Learning Curves')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('optuna_optimization_results.png', dpi=150)
plt.show()

print(f"\n💾 시각화 저장: optuna_optimization_results.png")

# ======================================================
# 최적 파라미터 저장
# ======================================================
with open('best_hyperparameters.json', 'w', encoding='utf-8') as f:
    json.dump({
        'best_value': study.best_value,
        'best_params': study.best_params,
        'n_trials': len(study.trials)
    }, f, indent=2, ensure_ascii=False)

print(f"💾 최적 파라미터 저장: best_hyperparameters.json")
print("="*70)

In [ ]:
# ======================================================
# 🚀 최적 하이퍼파라미터로 최종 학습
# ======================================================

print("="*70)
print("🚀 최적 하이퍼파라미터로 최종 모델 학습")
print("="*70)

# 최적 파라미터 가져오기
best_params = study.best_params
print(f"\n📌 사용할 하이퍼파라미터:")
for key, value in best_params.items():
    print(f"  - {key}: {value}")

# 파라미터 적용
NUM_GAUSSIANS_OPT = best_params['num_gaussians']
HIDDEN_DIM_OPT = best_params['hidden_dim']
LSTM_LAYERS_OPT = best_params['lstm_layers']
DROPOUT_OPT = best_params['dropout']
LR_OPT = best_params['lr']
BATCH_SIZE_OPT = best_params['batch_size']
MIN_SIGMA_OPT = best_params['min_sigma']
MAX_SIGMA_OPT = best_params['max_sigma']
HYBRID_LOSS_WEIGHT_OPT = best_params['hybrid_loss_weight']

# 3개 seed로 앙상블 학습
SEEDS_OPT = [42, 2024, 777]
EPOCHS_OPT = 50

all_histories_opt = []

for i, seed in enumerate(SEEDS_OPT):
    print(f"\n{'='*70}")
    print(f"📦 [최적 모델 {i+1}/3] Seed {seed}")
    print(f"{'='*70}")
    seed_everything(seed)
    
    train_idx, val_idx = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=seed)
    train_loader = DataLoader(torch.utils.data.Subset(full_dataset, train_idx), 
                              batch_size=BATCH_SIZE_OPT, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(torch.utils.data.Subset(full_dataset, val_idx), 
                           batch_size=BATCH_SIZE_OPT, shuffle=False, collate_fn=collate_fn)
    
    # 최적 파라미터로 모델 생성
    model_opt = ImprovedMDNPredictor(
        INPUT_DIM, HIDDEN_DIM_OPT, LSTM_LAYERS_OPT, DROPOUT_OPT, 
        NUM_GAUSSIANS_OPT, BIDIRECTIONAL
    ).to(DEVICE)
    
    optimizer = optim.AdamW(model_opt.parameters(), lr=LR_OPT, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS_OPT - WARMUP_EPOCHS, eta_min=1e-6)
    
    best_dist = float('inf')
    history = {'train_loss': [], 'val_dist': [], 'lr': []}
    
    for epoch in range(EPOCHS_OPT):
        # Warmup
        if epoch < WARMUP_EPOCHS:
            for param_group in optimizer.param_groups:
                param_group['lr'] = LR_OPT * (epoch + 1) / WARMUP_EPOCHS
        
        # Training
        model_opt.train()
        train_loss = 0.0
        for seqs, targets, mask, lens, _ in train_loader:
            seqs, targets, mask, lens = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
            optimizer.zero_grad()
            pi, mu, sigma = model_opt(seqs, mask, lens)
            
            # 최적 sigma 범위 적용
            sigma = torch.clamp(sigma, MIN_SIGMA_OPT, MAX_SIGMA_OPT)
            
            # 최적 loss weight 적용
            target_expanded = targets.unsqueeze(1).expand_as(mu)
            diff = target_expanded - mu
            log_prob_components = (
                -0.5 * math.log(2 * math.pi) - torch.log(sigma) - 0.5 * (diff / sigma) ** 2
            )
            log_prob = log_prob_components.sum(dim=2)
            weighted_log_prob = log_prob + torch.log(pi + 1e-8)
            nll_loss = -torch.logsumexp(weighted_log_prob, dim=1).mean()
            
            pred_mean = (pi.unsqueeze(-1) * mu).sum(dim=1)
            mse_loss = nn.functional.mse_loss(pred_mean, targets)
            
            loss = nll_loss + HYBRID_LOSS_WEIGHT_OPT * mse_loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model_opt.parameters(), GRAD_CLIP)
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model_opt.eval()
        dists = []
        with torch.no_grad():
            for seqs, targets, mask, lens, _ in val_loader:
                seqs, targets, mask, lens = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
                pi, mu, sigma = model_opt(seqs, mask, lens)
                pred = mdn_predict_improved(pi, mu, sigma, strategy='mean')
                p_real = pred.cpu().numpy() * np.array([105.0, 68.0])
                t_real = targets.cpu().numpy() * np.array([105.0, 68.0])
                dists.extend(np.sqrt(np.sum((p_real - t_real)**2, axis=1)))
        
        avg_dist = np.mean(dists)
        avg_loss = train_loss / len(train_loader)
        
        if epoch >= WARMUP_EPOCHS:
            scheduler.step()
        
        current_lr = optimizer.param_groups[0]['lr']
        history['train_loss'].append(avg_loss)
        history['val_dist'].append(avg_dist)
        history['lr'].append(current_lr)
        
        if (epoch + 1) % 5 == 0 or epoch < 5:
            print(f"  [Epoch {epoch+1:2d}/{EPOCHS_OPT}] "
                  f"Loss: {avg_loss:.4f} | Val: {avg_dist:.4f}m | LR: {current_lr:.6f}")
        
        if avg_dist < best_dist:
            best_dist = avg_dist
            torch.save(model_opt.state_dict(), f'optimized_mdn_{i}.pth')
    
    history['best_dist'] = best_dist
    history['seed'] = seed
    all_histories_opt.append(history)
    print(f"  ✅ Best: {best_dist:.4f}m")

print(f"\n{'='*70}")
print(f"✅ 최적화된 모델 학습 완료!")
print(f"{'='*70}")
print(f"평균 Best: {np.mean([h['best_dist'] for h in all_histories_opt]):.4f}m")
print(f"최고 성능: {min([h['best_dist'] for h in all_histories_opt]):.4f}m")
print(f"기존 V11 대비 개선: {14.46 - np.mean([h['best_dist'] for h in all_histories_opt]):.4f}m")
print(f"{'='*70}")